### **POS Tagging using CRF Model**

In [ ]:
pip install sklearn-crfsuite

In [ ]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
pip install pyconll

Building the training and test set

In [ ]:
import pyconll
from pathlib import Path
from conllu import parse
from io import open
train= Path('/content/hi-ud-train.conllu').read_text()

In [ ]:
train

"ID,WORD,POS_TAG\n1,yaha,DET\n2,eSiyA,PROPN\n3,kI,ADP\n4,sabase,ADV\n5,badZI,ADJ\n6,masjixoM,NOUN\n7,meM,ADP\n8,se,ADP\n9,eka,NUM\n10,hE,AUX\n11,.,PUNCT\n,,\n1,ise,PRON\n2,navAba,NOUN\n3,SAhajehana,PROPN\n4,ne,ADP\n5,banavAyA,VERB\n6,WA,AUX\n7,.,PUNCT\n,,\n1,isakA,PRON\n2,praveSa,NOUN\n3,xvAra,NOUN\n4,xo,NUM\n5,maMjilA,ADJ\n6,hE,AUX\n7,.,PUNCT\n,,\n1,jisameM,PRON\n2,cAra,NUM\n3,meharAbeM,NOUN\n4,hEM,AUX\n5,Ora,CCONJ\n6,muKya,ADJ\n7,prArWanA,NOUN\n8,hOYla,NOUN\n9,meM,ADP\n10,jAne,VERB\n11,ke,ADP\n12,lie,ADP\n13,9,NUM\n14,praveSa,NOUN\n15,xvAra,NOUN\n16,hEM,VERB\n17,.,PUNCT\n,,\n1,pUrI,ADJ\n2,imArawa,NOUN\n3,behaxa,ADV\n4,KUbasUrawa,ADJ\n5,hE,AUX\n6,.,PUNCT\n,,\n1,yahAz,PRON\n2,lagane,VERB\n3,vAlA,ADP\n4,wIna,NUM\n5,xina,NOUN\n6,kA,ADP\n7,ijwimA,NOUN\n8,pUre,ADJ\n9,xeSa,NOUN\n10,ke,ADP\n11,logoM,NOUN\n12,ko,ADP\n13,AmaMwriwa,ADJ\n14,karawA,VERB\n15,hE,AUX\n16,.,PUNCT\n,,\n1,SOkawa,PROPN\n2,mahala,PROPN\n3,ke,ADP\n4,sAmane,ADP\n5,badZI,ADJ\n6,JIla,NOUN\n7,ke,ADP\n8,kinAre,NOUN\n9,sWiwa,AD

In [ ]:
records= train.splitlines()

In [ ]:
train_sent=[]
i=1
while(i<len(records)):
  sent=[]
  j=i
  vals=records[j].split(',')
  while(vals[0]!=''):
    sent.append((vals[1],vals[2]))
    j=j+1
    if(j>=len(records)):
      break
    vals=records[j].split(',')
  train_sent.append(sent)
  i=j+1

In [ ]:
print(train_sent)

[[('yaha', 'DET'), ('eSiyA', 'PROPN'), ('kI', 'ADP'), ('sabase', 'ADV'), ('badZI', 'ADJ'), ('masjixoM', 'NOUN'), ('meM', 'ADP'), ('se', 'ADP'), ('eka', 'NUM'), ('hE', 'AUX'), ('.', 'PUNCT')], [('ise', 'PRON'), ('navAba', 'NOUN'), ('SAhajehana', 'PROPN'), ('ne', 'ADP'), ('banavAyA', 'VERB'), ('WA', 'AUX'), ('.', 'PUNCT')], [('isakA', 'PRON'), ('praveSa', 'NOUN'), ('xvAra', 'NOUN'), ('xo', 'NUM'), ('maMjilA', 'ADJ'), ('hE', 'AUX'), ('.', 'PUNCT')], [('jisameM', 'PRON'), ('cAra', 'NUM'), ('meharAbeM', 'NOUN'), ('hEM', 'AUX'), ('Ora', 'CCONJ'), ('muKya', 'ADJ'), ('prArWanA', 'NOUN'), ('hOYla', 'NOUN'), ('meM', 'ADP'), ('jAne', 'VERB'), ('ke', 'ADP'), ('lie', 'ADP'), ('9', 'NUM'), ('praveSa', 'NOUN'), ('xvAra', 'NOUN'), ('hEM', 'VERB'), ('.', 'PUNCT')], [('pUrI', 'ADJ'), ('imArawa', 'NOUN'), ('behaxa', 'ADV'), ('KUbasUrawa', 'ADJ'), ('hE', 'AUX'), ('.', 'PUNCT')], [('yahAz', 'PRON'), ('lagane', 'VERB'), ('vAlA', 'ADP'), ('wIna', 'NUM'), ('xina', 'NOUN'), ('kA', 'ADP'), ('ijwimA', 'NOUN'), (

In [ ]:
test= Path('/content/hi-ud-test .conllu').read_text()

In [ ]:
test

'ID\tWORD\tTAG\n1\trAmAyaNa\tPROPN\n2\tkAla\tPROPN\n3\tmeM\tADP\n4\tBagavAna\tNOUN\n5\trAma\tPROPN\n6\tke\tADP\n7\tpuwra\tNOUN\n8\tkuSa\tPROPN\n9\tkI\tADP\n10\trAjaXAnI\tNOUN\n11\tkuSAvawI\tPROPN\n12\tko\tADP\n13\t483\tPROPN\n14\tIsA\tPROPN\n15\tpUrva\tPROPN\n16\tbuxXa\tPROPN\n17\tne\tADP\n18\tapane\tPRON\n19\taMwima\tADJ\n20\tviSrAma\tNOUN\n21\tke\tADP\n22\tlie\tADP\n23\tcunA\tVERB\n24\t.\tPUNCT\n\t\t\n1\tmalloM\tNOUN\n2\tkI\tADP\n3\trAjaXAnI\tNOUN\n4\thone\tVERB\n5\tke\tADP\n6\tkAraNa\tADP\n7\tprAcInakAla\tNOUN\n8\tmeM\tADP\n9\tisa\tDET\n10\tsWAna\tNOUN\n11\tkA\tADP\n12\tawyaMwa\tADJ\n13\tmahawva\tNOUN\n14\tWA\tVERB\n15\t.\tPUNCT\n\t\t\n1\tbOxXa\tPROPN\n2\tXarmAvalaMbiyoM\tNOUN\n3\tke\tADP\n4\tanusAra\tADP\n5\tluMbanI\tPROPN\n6\t","\tPUNCT\n7\tboXagayA\tPROPN\n8\tOra\tCCONJ\n9\tsAranAWa\tPROPN\n10\tke\tADP\n11\tsAWa\tADP\n12\thI\tPART\n13\tisa\tDET\n14\tsWAna\tNOUN\n15\tkA\tADP\n16\tviSax\tADJ\n17\tmahawva\tNOUN\n18\thE\tAUX\n19\t.\tPUNCT\n\t\t\n1\thiMxU\tADJ\n2\trAjAoM\tNOUN\n3\tke\

In [ ]:
test_records=test.splitlines()

In [ ]:
test_sent=[]
i=1
while(i<len(test_records)):
  sent=[]
  j=i
  vals=test_records[j].split('\t')
  while(vals[0]!=''):
    sent.append((vals[1],vals[2]))
    j=j+1
    if(j>=len(test_records)):
      break
    vals=test_records[j].split('\t')
  test_sent.append(sent)
  i=j+1

In [ ]:
print(test_sent)

[[('rAmAyaNa', 'PROPN'), ('kAla', 'PROPN'), ('meM', 'ADP'), ('BagavAna', 'NOUN'), ('rAma', 'PROPN'), ('ke', 'ADP'), ('puwra', 'NOUN'), ('kuSa', 'PROPN'), ('kI', 'ADP'), ('rAjaXAnI', 'NOUN'), ('kuSAvawI', 'PROPN'), ('ko', 'ADP'), ('483', 'PROPN'), ('IsA', 'PROPN'), ('pUrva', 'PROPN'), ('buxXa', 'PROPN'), ('ne', 'ADP'), ('apane', 'PRON'), ('aMwima', 'ADJ'), ('viSrAma', 'NOUN'), ('ke', 'ADP'), ('lie', 'ADP'), ('cunA', 'VERB'), ('.', 'PUNCT')], [('malloM', 'NOUN'), ('kI', 'ADP'), ('rAjaXAnI', 'NOUN'), ('hone', 'VERB'), ('ke', 'ADP'), ('kAraNa', 'ADP'), ('prAcInakAla', 'NOUN'), ('meM', 'ADP'), ('isa', 'DET'), ('sWAna', 'NOUN'), ('kA', 'ADP'), ('awyaMwa', 'ADJ'), ('mahawva', 'NOUN'), ('WA', 'VERB'), ('.', 'PUNCT')], [('bOxXa', 'PROPN'), ('XarmAvalaMbiyoM', 'NOUN'), ('ke', 'ADP'), ('anusAra', 'ADP'), ('luMbanI', 'PROPN'), ('","', 'PUNCT'), ('boXagayA', 'PROPN'), ('Ora', 'CCONJ'), ('sAranAWa', 'PROPN'), ('ke', 'ADP'), ('sAWa', 'ADP'), ('hI', 'PART'), ('isa', 'DET'), ('sWAna', 'NOUN'), ('kA', '

Defining features

In [ ]:
def word2features(sentence, index):
    word=sentence[index][0]
    return {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        # 'is_capitalized': word[0].upper() == word[0],
        'is_all_caps': word.upper() == word,
        'is_all_lower': word.lower() == word,
        # 'prefix-1': word[0],
        # 'prefix-2': word[:2],
        # 'suffix-1': word[-1],
        # 'suffix-2': word[-2:],
        'prev_word': '' if index == 0 else sentence[index - 1][0],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1][0],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
    }

Features

*   'word' feature contains the word of sentence at index 'index' of the corpus
*   'is_first' checks if the word is the first word of the sentence

*   'is_last' checks if the word is the last word of the sentence
*   'is_all_caps' checks if word is in uppercase

*   'is_all_lower' checks if word is in lowercase
*   'prev_word' makes the previous word of the sentence as a feature if it exists

*   'next_word' makes the next word of the sentence as a feature if it exists
*   'has_hyphen' checks if the word has a hyphen

*   'is_numeric' checks if the word is a numerical value













In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [postag for token, postag in sent]

In [ ]:
X_train = [sent2features(s) for s in train_sent]

In [ ]:
y_train = [sent2labels(s) for s in train_sent]

In [ ]:
print(X_train)

[[{'word': 'yaha', 'is_first': True, 'is_last': False, 'is_all_caps': False, 'is_all_lower': True, 'prev_word': '', 'next_word': 'eSiyA', 'has_hyphen': False, 'is_numeric': False}, {'word': 'eSiyA', 'is_first': False, 'is_last': False, 'is_all_caps': False, 'is_all_lower': False, 'prev_word': 'yaha', 'next_word': 'kI', 'has_hyphen': False, 'is_numeric': False}, {'word': 'kI', 'is_first': False, 'is_last': False, 'is_all_caps': False, 'is_all_lower': False, 'prev_word': 'eSiyA', 'next_word': 'sabase', 'has_hyphen': False, 'is_numeric': False}, {'word': 'sabase', 'is_first': False, 'is_last': False, 'is_all_caps': False, 'is_all_lower': True, 'prev_word': 'kI', 'next_word': 'badZI', 'has_hyphen': False, 'is_numeric': False}, {'word': 'badZI', 'is_first': False, 'is_last': False, 'is_all_caps': False, 'is_all_lower': False, 'prev_word': 'sabase', 'next_word': 'masjixoM', 'has_hyphen': False, 'is_numeric': False}, {'word': 'masjixoM', 'is_first': False, 'is_last': False, 'is_all_caps': Fal

In [ ]:
print(y_train)

[['DET', 'PROPN', 'ADP', 'ADV', 'ADJ', 'NOUN', 'ADP', 'ADP', 'NUM', 'AUX', 'PUNCT'], ['PRON', 'NOUN', 'PROPN', 'ADP', 'VERB', 'AUX', 'PUNCT'], ['PRON', 'NOUN', 'NOUN', 'NUM', 'ADJ', 'AUX', 'PUNCT'], ['PRON', 'NUM', 'NOUN', 'AUX', 'CCONJ', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'VERB', 'ADP', 'ADP', 'NUM', 'NOUN', 'NOUN', 'VERB', 'PUNCT'], ['ADJ', 'NOUN', 'ADV', 'ADJ', 'AUX', 'PUNCT'], ['PRON', 'VERB', 'ADP', 'NUM', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADJ', 'VERB', 'AUX', 'PUNCT'], ['PROPN', 'PROPN', 'ADP', 'ADP', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PROPN', 'PROPN', 'ADP', 'NOUN', 'ADP', 'AUX', 'PRON', 'PROPN', 'PROPN', 'PART', 'VERB', 'AUX', 'AUX', 'PUNCT'], ['PRON', 'NOUN', 'CCONJ', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'AUX', 'PUNCT'], ['PRON', 'PROPN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'AUX', 'PUNCT'], ['ADJ', 'NOUN', 'ADP', 'PRON', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'AUX', 'PUNCT'], ['PRON', 'NOUN', 'PROPN', 'AD

In [ ]:
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

Training the CRF Model

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Evaluation

In [ ]:
labels = list(crf.classes_)
labels

['DET',
 'PROPN',
 'ADP',
 'ADV',
 'ADJ',
 'NOUN',
 'NUM',
 'AUX',
 'PUNCT',
 'PRON',
 'VERB',
 'CCONJ',
 'PART',
 'COMMA',
 'SCONJ',
 'X']

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.802887030769118

In [ ]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      1.000     0.879     0.935        33
       CCONJ      1.000     1.000     1.000        25
       SCONJ      0.667     0.667     0.667         3
         ADJ      0.526     0.649     0.581        94
         ADP      0.961     0.958     0.959       309
         ADV      0.692     0.429     0.529        21
        VERB      0.747     0.747     0.747        99
         DET      0.744     0.806     0.773        36
       COMMA      0.000     0.000     0.000         0
        NOUN      0.716     0.836     0.771       329
        PRON      0.896     0.662     0.761        65
       PROPN      0.580     0.352     0.438       145
         NUM      0.846     0.880     0.863        25
       PUNCT      1.000     0.963     0.981       135
         AUX      0.887     0.957     0.920       139

   micro avg      0.809     0.809     0.809      1458
   macro avg      0.704   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Hyperparameter optimization

In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc0376ba2b0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc0376ba470>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,


Best estimator

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.06460620845136053, 'c2': 0.017467791192444133}
best CV score: 0.8144392880335608
model size: 0.29M


Classification Report

Train data

In [ ]:
crf = rs.best_estimator_
y_pred_train = crf.predict(X_train)
print(metrics.flat_classification_report(
    y_train, y_pred_train, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      1.000     1.000     1.000         2
        PART      1.000     1.000     1.000       163
       CCONJ      1.000     1.000     1.000       150
       SCONJ      1.000     1.000     1.000        61
         ADJ      1.000     1.000     1.000       570
         ADP      0.999     1.000     1.000      1387
         ADV      1.000     0.991     0.995       111
        VERB      1.000     0.992     0.996       640
         DET      1.000     1.000     1.000       231
       COMMA      1.000     1.000     1.000       114
        NOUN      0.999     1.000     1.000      1597
        PRON      1.000     1.000     1.000       431
       PROPN      1.000     0.999     0.999       708
         NUM      1.000     1.000     1.000       152
       PUNCT      1.000     1.000     1.000       564
         AUX      0.993     1.000     0.997       730

    accuracy                          0.999      7611
   macro avg      0.999   

In [ ]:
print("F1 Score(train data): ")
print(metrics.flat_f1_score(y_train, y_pred_train, 
                      average='weighted', labels=labels))

F1 Score(train data): 
0.9990798120320605


In [ ]:
print('Accuracy(train data): ')
print(metrics.flat_accuracy_score(y_train, y_pred_train))

Accuracy(train data): 
0.9990802785442123


Test data

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      1.000     0.909     0.952        33
       CCONJ      1.000     1.000     1.000        25
       SCONJ      0.667     0.667     0.667         3
         ADJ      0.571     0.638     0.603        94
         ADP      0.968     0.971     0.969       309
         ADV      0.769     0.476     0.588        21
        VERB      0.770     0.778     0.774        99
         DET      0.696     0.889     0.780        36
       COMMA      0.000     0.000     0.000         0
        NOUN      0.738     0.866     0.797       329
        PRON      0.863     0.677     0.759        65
       PROPN      0.567     0.352     0.434       145
         NUM      0.852     0.920     0.885        25
       PUNCT      1.000     0.926     0.962       135
         AUX      0.912     0.964     0.937       139

   micro avg      0.822     0.822     0.822      1458
   macro avg      0.711   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("F1 Score(test data): ")
print(metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels))

F1 Score(test data): 
0.815083651455432


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [ ]:
print('Accuracy(test data): ')
print(metrics.flat_accuracy_score(y_test, y_pred))

Accuracy(test data): 
0.821673525377229


Top 10 most likely transitions

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(10))

Top likely transitions:
ADJ    -> NOUN    4.955808
VERB   -> AUX     4.682279
PROPN  -> PROPN   4.164626
NOUN   -> VERB    3.366872
ADJ    -> VERB    3.231638
NUM    -> NOUN    3.190743
DET    -> NOUN    2.942022
NOUN   -> ADP     2.889874
NOUN   -> NOUN    2.652111
PROPN  -> ADP     2.577988


In [ ]:
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-10:])


Top unlikely transitions:
ADP    -> CCONJ   -1.897117
COMMA  -> ADP     -1.905769
ADJ    -> ADP     -1.965273
ADP    -> COMMA   -2.154188
AUX    -> ADP     -2.186727
NUM    -> PRON    -2.189970
DET    -> CCONJ   -2.210640
CCONJ  -> PART    -2.235781
CCONJ  -> AUX     -2.363753
DET    -> ADP     -3.589769
